In [1]:
import tweepy
import pickle
import pandas as pd
import numpy as np
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [39]:
filename = 'data.p'
with open(filename, 'rb') as f:
    classification_dict = pickle.load(f, encoding='bytes')

In [195]:
emotions_dict = {
    'joy': np.array([1., 0., 0., 0., 0., 0., 0.]),
    'fear': np.array([0., 1., 0., 0., 0., 0., 0.]),
    'anger': np.array([0., 0., 1., 0., 0., 0., 0.]),
    'sadness': np.array([0., 0., 0., 1., 0., 0., 0.]),
    'disgust': np.array([0., 0., 0., 0., 1., 0., 0.]),
    'shame': np.array([0., 0., 0., 0., 0., 1., 0.]),
    'guilt': np.array([0., 0., 0., 0., 0., 0., 1.])
}
i=0
np.array_equal(list(list(classification_dict.values())[0][i].values())[0], list(emotions_dict.items())[0][1])

True

In [257]:
def create_dataframe(dataset):
    i=0
    for key, val in list(dataset.keys()):
        print(list(dataset[key])[i])
        i+=1
# create_dataframe(classification_dict)
a = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_dict.items() ]))

In [5]:
df = pd.read_csv('data_emotions.csv')
text_list = np.array(list(df.text))
text_list = [x.split() for x in text_list]
text_list[0]

['During',
 'the',
 'period',
 'of',
 'falling',
 'in',
 'love,',
 'each',
 'time',
 'that',
 'we',
 'met',
 'and',
 'especially',
 'when',
 'we',
 'had',
 'not',
 'met',
 'for',
 'a',
 'long',
 'time.']

In [6]:
model = Word2Vec(text_list)
model.save("word2vec.model")

In [13]:
words = list(model.wv.vocab)
print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

I the a


In [14]:
def convert_data_to_index(string_data, wv):
    index_data = []
    for word in string_data:
        if word in wv:
            index_data.append(wv.vocab[word].index)
    return index_data

In [20]:
print(model.wv.most_similar('dirty'))

[('getting', 0.999699592590332), ('against', 0.9996154308319092), ('following', 0.999600887298584), ('persons', 0.9995965957641602), ('rather', 0.9995722770690918), ('making', 0.9995700120925903), ("I'd", 0.9995676279067993), ('instead', 0.9995375871658325), ('Then', 0.9995261430740356), ('way.', 0.9995232820510864)]


C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

In [30]:
df.emotion.shape

(7480,)

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(df.text, df.emotion,
                    batch_size=100,
                    epochs=100,
                    verbose=1)

ValueError: Error when checking input: expected dense_3_input to have shape (784,) but got array with shape (1,)

In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [35]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [40]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 171, 128)          256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 171, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 1379      
Total params: 512,179
Trainable params: 512,179
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
Y = pd.get_dummies(df['emotion']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5011, 171) (5011, 7)
(2469, 171) (2469, 7)


In [42]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 46s - loss: 1.8872 - acc: 0.2371
Epoch 2/7
 - 48s - loss: 1.5160 - acc: 0.4442
Epoch 3/7
 - 51s - loss: 1.2051 - acc: 0.5737
Epoch 4/7
 - 53s - loss: 0.9936 - acc: 0.6510
Epoch 5/7
 - 53s - loss: 0.8709 - acc: 0.6967
Epoch 6/7
 - 50s - loss: 0.7615 - acc: 0.7416
Epoch 7/7
 - 46s - loss: 0.6904 - acc: 0.7633


In [84]:
twt = ['i feel sad']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=171, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)
arg_max = np.argmax(sentiment)
if arg_max == 0:
    print('joy')
elif arg_max == 1:
    print('fear')
elif arg_max == 2:
    print('anger')
elif arg_max == 3:
    print('sadness')
elif arg_max == 4:
    print('disgust')
elif arg_max == 5:
    print('shame')
elif arg_max == 6:
    print('guilt')

[1.5083022e-04 8.1259714e-05 2.0514749e-04 2.1620069e-03 2.8652127e-04
 9.9692518e-01 1.8913335e-04]
shame


In [63]:
validation_size = 100

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.37
acc: 0.55
